In [1]:
#Jia Yi (Susan) Wang
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import re
#import seaborn
#seaborn.set()

from sklearn.model_selection import train_test_split
#from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

""""from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC"""

"""from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier"""

#from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
""""from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor"""
from sklearn.linear_model import LinearRegression

#from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import TruncatedSVD #PCA for sparse values
from sklearn.ensemble import GradientBoostingClassifier

# Getting the data we need

In [2]:
genres = pd.read_json('data/genres.json.gz', orient='record', lines=True, encoding='utf-8')
omdb_data = pd.read_json('data/omdb-data.json.gz', orient='record', lines=True, encoding='utf-8')
rt_data = pd.read_json('data/rotten-tomatoes.json.gz', orient='record', lines=True, encoding='utf-8')
wd_data = pd.read_json('data/wikidata-movies.json.gz', orient='record', lines=True, encoding='utf-8')

In [3]:
omdb_data=omdb_data.explode('omdb_genres')
omdb_data=omdb_data[omdb_data.omdb_awards.notnull()]
omdb_data=omdb_data[omdb_data.omdb_awards!='N\A']

nominations_re = re.compile(r'Nominated for (\d+)')
nominations_re2 = re.compile(r'(\d+) nomination(s?)')
wins_re = re.compile(r'(\d+) win(s?)')
wins_re2 = re.compile(r'Won (\d+)')

def get_wins(txt):
    wins = 0
    wins1 = wins_re.search(txt)
    wins2 = wins_re2.search(txt)
    if wins1:
        wins = wins + int(wins1.group(1))
    if wins2:
        wins = wins + int(wins2.group(1))
    return wins

def get_nominations(txt):
    noms = 0
    nominations1 = nominations_re.search(txt)
    nominations2 = nominations_re2.search(txt)
    if nominations1:
        noms = noms + int(nominations1.group(1))
    if nominations2:
        noms = noms + int(nominations2.group(1))
    return noms

omdb_data['nominations'] = omdb_data['omdb_awards'].apply(get_nominations)
omdb_data['wins'] = omdb_data['omdb_awards'].apply(get_wins)
    

In [4]:
#np.set_printoptions(threshold=np.inf)
#set(omdb_data.omdb_awards)
omdb_data

,imdb_id,omdb_genres,omdb_plot,omdb_awards,nominations,wins
0,tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0
0,tt0060814,History,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0
0,tt0060814,War,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0
1,tt2379713,Action,A cryptic message from the past sends James Bo...,Won 1 Oscar. Another 7 wins & 32 nominations.,32,8
1,tt2379713,Adventure,A cryptic message from the past sends James Bo...,Won 1 Oscar. Another 7 wins & 32 nominations.,32,8
1,tt2379713,Thriller,A cryptic message from the past sends James Bo...,Won 1 Oscar. Another 7 wins & 32 nominations.,32,8
2,tt0230575,Comedy,"The makers of this parody of ""Night of the Liv...",N/A,0,0
2,tt0230575,Horror,"The makers of this parody of ""Night of the Liv...",N/A,0,0
3,tt0065988,Adventure,Jack Crabb is 121 years old as the film begins...,Nominated for 1 Oscar. Another 5 wins & 8 nomi...,9,5
3,tt0065988,Comedy,Jack Crabb is 121 years old as the film begins...,Nominated for 1 Oscar. Another 5 wins & 8 nomi...,9,5


In [5]:
wd_data = wd_data.explode('cast_member')
movies_played=wd_data[['cast_member','wikidata_id']].groupby('cast_member').count().reset_index().rename(columns={'wikidata_id':'movies_in'})
wd_data_stars = movies_played[movies_played.movies_in>=10]
wd_data_stars = wd_data_stars.set_index('cast_member').join(wd_data.set_index('cast_member'),on='cast_member').reset_index()

In [6]:
wd_data_stars #filter out actors who < 10 movies

,cast_member,movies_in,wikidata_id,label,imdb_id,rotten_tomatoes_id,enwiki_title,genre,director,publication_date,country_of_origin,original_language,filming_location,based_on,made_profit,metacritic_id,main_subject,series
0,Q1001214,11,Q1211343,The Beverly Hillbillies,tt0106400,m/1046435-beverly_hillbillies,The Beverly Hillbillies (film),[Q157443],[Q441578],1993-01-01,Q30,Q1860,[Q3476090],NaN,NaN,NaN,NaN,NaN
1,Q1001214,11,Q1619460,Broadway Melody of 1936,tt0026144,m/broadway_melody_of_1936,Broadway Melody of 1936,[Q842256],"[Q918762, Q547078]",1935-01-01,Q30,Q1860,NaN,NaN,NaN,NaN,NaN,NaN
2,Q1001214,11,Q193066,Breakfast at Tiffany's,tt0054698,m/breakfast_at_tiffanys,Breakfast at Tiffany's (film),"[Q860626, Q157443, Q52162262]",[Q56093],1961-01-01,Q30,Q1860,[Q60],Q849930,1.0,movie/breakfast-at-tiffanys,NaN,NaN
3,Q1001214,11,Q229547,Born to Dance,tt0027387,m/born_to_dance,Born to Dance,[Q842256],[Q918762],1936-01-01,Q30,Q1860,NaN,NaN,NaN,NaN,NaN,NaN
4,Q1001214,11,Q1093219,"Davy Crockett, King of the Wild Frontier",tt0047977,m/davy_crockett_king_of_the_wild_frontier,"Davy Crockett, King of the Wild Frontier",[Q2143665],[Q2409726],1955-05-25,Q30,Q1860,"[Q99, Q1454]",NaN,NaN,NaN,[Q638530],Q3019192
5,Q1001214,11,Q1424447,The Girl of the Golden West,tt0030182,m/girl_of_the_golden_west,The Girl of the Golden West (1938 film),"[Q130232, Q1054574, Q842256, Q172980]",[Q269731],1938-01-01,Q30,Q1860,NaN,NaN,NaN,NaN,NaN,NaN
6,Q1001214,11,Q624658,Mail Order Bride,tt0058318,m/771070746,Mail Order Bride (1964 film),[Q172980],[Q786712],1964-01-01,Q30,Q1860,NaN,NaN,NaN,NaN,NaN,NaN
7,Q1001214,11,Q3202604,"The One and Only, Genuine, Original Family Band",tt0063389,m/one_and_only_genuine_original_family_band,"The One and Only, Genuine, Original Family Band",[Q842256],[Q1688887],1968-03-21,Q30,Q1860,NaN,NaN,NaN,NaN,NaN,NaN
8,Q1001214,11,Q3282128,Red Garters,tt0047400,m/red_garters,Red Garters (film),[Q842256],[Q1384550],1954-01-01,Q30,Q1860,NaN,NaN,NaN,NaN,NaN,NaN
9,Q1001214,11,Q7522737,Sing Your Worries Away,tt0035338,m/sing_your_worries_away,Sing Your Worries Away,[Q842256],[Q278657],1942-01-01,Q30,Q1860,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
joined = omdb_data.set_index('imdb_id').join(rt_data.set_index('imdb_id'),on='imdb_id')
joined = joined.join(wd_data_stars.set_index('imdb_id'), lsuffix='_joined', rsuffix='_wd', on='imdb_id')

In [8]:
joined

,omdb_genres,omdb_plot,omdb_awards,nominations,wins,audience_average,audience_percent,audience_ratings,critic_average,critic_percent,...,director,publication_date,country_of_origin,original_language,filming_location,based_on,made_profit,metacritic_id,main_subject,series
imdb_id,,,,,,,,,,,,,,,,,,,,,
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN


# Defining our Features and Labels and filtering data

In [9]:
info = joined.reset_index()
info = info[['audience_average', 'omdb_genres', 'cast_member', 'wins', 'nominations', 'country_of_origin']]
info = info[info.audience_average.notnull()]
info = info[info.cast_member.notnull()]
info = info[info.omdb_genres.notnull()]
info = info[info.omdb_genres != 'N/A']
info = info[info.country_of_origin.notnull()]
info = info[info.country_of_origin != 'N/A']

# Encoding categorical values

In [10]:
#One hot encoder example: https://medium.com/@contactsunny/label-encoder-vs-one-hot-encoder-in-machine-learning-3fc273365621
#Column Transforer source: https://datascience.stackexchange.com/questions/41113/deprecationwarning-the-categorical-features-keyword-is-deprecated-in-version

#Encoding the genres to binary

ct = ColumnTransformer(
    transformers=[
        ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), [0,1,4])
    ],
    remainder='passthrough' 
) #randomforestregressor - try this

#X = ct.fit_transform(X)

In [11]:
info

,audience_average,omdb_genres,cast_member,wins,nominations,country_of_origin
0,3.5,Drama,Q104027,0,4,Q142
1,3.5,Drama,Q105987,0,4,Q142
2,3.5,Drama,Q106057,0,4,Q142
3,3.5,Drama,Q106126,0,4,Q142
4,3.5,Drama,Q106204,0,4,Q142
5,3.5,Drama,Q106255,0,4,Q142
6,3.5,Drama,Q106529,0,4,Q142
7,3.5,Drama,Q1096611,0,4,Q142
8,3.5,Drama,Q1097113,0,4,Q142
9,3.5,Drama,Q1358541,0,4,Q142


# GradientBoostingRegessor model for predicting audience average

In [12]:
model = Pipeline(
        #steps=[('col_trans',ct), ('lr', LinearRegression(fit_intercept=False))]
        steps=[
            ('col_trans',ct), 
            ('pca', TruncatedSVD(2)), 
            ('gfr', GradientBoostingRegressor(max_depth=5, n_estimators=100, min_samples_leaf=10))
        ]
    )

X=info.drop(columns=['audience_average'],axis=1)
y=info['audience_average']

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

model.fit(X_train, y_train)

print(model.score(X_train,y_train))
print(model.score(X_valid,y_valid)) 

#For actors that starred in >1 movie:
#m_d=5: train=0.3398654, valid=0.33893729 15.3 s
#m_d=6: train=0.37, valid=0.36 20.3s
#m_d=7: train=0.40, valid=0.39 23.3s
#m_d=8: train=0.42, valid=0.40 30.7s
#m_d=9: train=0.45, valid=0.43 34.1s
#m_d=10 : train=0.46, valid=0.44 44.4s
#m_d=15 : train=0.53, valid=0.47 1min37s
#m_d=20 : train=0.55, valid=0.47 2min55s
#m_d=30 : train=0.57, valid=0.46 4min48s

0.35703702193772213
0.3537020119887683


# Same model for predicting critic average

In [13]:
info = joined.reset_index()
info = info[['critic_average', 'omdb_genres', 'cast_member', 'wins', 'nominations', 'country_of_origin']]
info = info[info.critic_average.notnull()]
info = info[info.cast_member.notnull()]
info = info[info.omdb_genres.notnull()]
info = info[info.omdb_genres != 'N/A']
info = info[info.country_of_origin.notnull()]
info = info[info.country_of_origin != 'N/A']

ct = ColumnTransformer(
    transformers=[
        ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), [0,1,4])
    ],
    remainder='passthrough' 
)

model = Pipeline(
        #steps=[('col_trans',ct), ('lr', LinearRegression(fit_intercept=False))]
        steps=[
            ('col_trans',ct), 
            ('pca', TruncatedSVD(2)), 
            ('gfr', GradientBoostingRegressor(max_depth=5, n_estimators=100, min_samples_leaf=10))
        ]
    )

X=info.drop(columns=['critic_average'],axis=1)
y=info['critic_average']

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

model.fit(X_train, y_train)

print(model.score(X_train,y_train))
print(model.score(X_valid,y_valid)) 



0.4230949670692249
0.4147061010205805


# Same model for predicting audience percent

In [14]:
info = joined.reset_index()
info = info[['audience_percent', 'omdb_genres', 'cast_member', 'wins', 'nominations', 'country_of_origin']]
info = info[info.audience_percent.notnull()]
info = info[info.cast_member.notnull()]
info = info[info.omdb_genres.notnull()]
info = info[info.omdb_genres != 'N/A']
info = info[info.country_of_origin.notnull()]
info = info[info.country_of_origin != 'N/A']

ct = ColumnTransformer(
    transformers=[
        ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), [0,1,4])
    ],
    remainder='passthrough' 
)

model = Pipeline(
        #steps=[('col_trans',ct), ('lr', LinearRegression(fit_intercept=False))]
        steps=[
            ('col_trans',ct), 
            ('pca', TruncatedSVD(2)), 
            ('gfr', GradientBoostingRegressor(max_depth=5, n_estimators=100, min_samples_leaf=10))
        ]
    )

X=info.drop(columns=['audience_percent'],axis=1)
y=info['audience_percent']

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

model.fit(X_train, y_train)

print(model.score(X_train,y_train))
print(model.score(X_valid,y_valid)) 

0.3813345359279158
0.37684707418118524


# Same model for predicting critic percent

In [15]:
info = joined.reset_index()
info = info[['critic_percent', 'omdb_genres', 'cast_member', 'wins', 'nominations', 'country_of_origin']]
info = info[info.critic_percent.notnull()]
info = info[info.cast_member.notnull()]
info = info[info.omdb_genres.notnull()]
info = info[info.omdb_genres != 'N/A']
info = info[info.country_of_origin.notnull()]
info = info[info.country_of_origin != 'N/A']

ct = ColumnTransformer(
    transformers=[
        ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), [0,1,4])
    ],
    remainder='passthrough' 
)

model = Pipeline(
        #steps=[('col_trans',ct), ('lr', LinearRegression(fit_intercept=False))]
        steps=[
            ('col_trans',ct), 
            ('pca', TruncatedSVD(2)), 
            ('gfr', GradientBoostingRegressor(max_depth=5, n_estimators=100, min_samples_leaf=10))
        ]
    )

X=info.drop(columns=['critic_percent'],axis=1)
y=info['critic_percent']

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

model.fit(X_train, y_train)

print(model.score(X_train,y_train))
print(model.score(X_valid,y_valid)) 

0.35135723615734105
0.3369150264759809


# Same model for predicting if a movie profits

In [16]:
info = joined.reset_index()
info = info[['made_profit', 'omdb_genres', 'cast_member', 'wins', 'nominations', 'country_of_origin']]
info = info[info.made_profit.notnull()]
info = info[info.cast_member.notnull()]
info = info[info.omdb_genres.notnull()]
info = info[info.omdb_genres != 'N/A']
info = info[info.country_of_origin.notnull()]
info = info[info.country_of_origin != 'N/A']

ct = ColumnTransformer(
    transformers=[
        ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), [0,1,4])
    ],
    remainder='passthrough' 
)

model = Pipeline(
        #steps=[('col_trans',ct), ('lr', LinearRegression(fit_intercept=False))]
        steps=[
            ('col_trans',ct), 
            ('pca', TruncatedSVD(2)), 
            ('gfr', GradientBoostingClassifier(max_depth=5, n_estimators=100, min_samples_leaf=10))
        ]
    )

X=info.drop(columns=['made_profit'],axis=1)
y=info['made_profit']

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

model.fit(X_train, y_train)

print(model.score(X_train,y_train))
print(model.score(X_valid,y_valid)) 

0.9453778744152336
0.9349673848586677
